# Homework Assignment 15


## Instructions

Consider the reservoir shown below with the given properties that has been discretized into equal grid blocks.

![image](images/grid.png)

Use the class template below and inherit from your [Assignment 13](https://github.com/PGE323M-Fall2017/assignment13) `OneDimReservoir()` class, complete the implementation of `compute_transmissibility()` and `compute_accumulation()` such that they will compute interblock transmissiblities for grid blocks that could potentially differ in permeability, grid spacing in both the $x$ and $y$-directions and depth, as well as accumulation terms that can vary from grid block to grid block in terms of volumes and porosities. 

Then complete the implementation of `fill_matrices()` to account for the additional boundary conditions on the top and bottom. You may look back to [Assignment 4](https://github.com/PGE323M-Fall2017/assignment4) for help with the structure of the now, pentadiagonal matrix.

There are now a few additional inputs such as number of grid block in the $x$ and $y$ directions. See the [test.py](test.py) file for the additional input syntax.  Your code should still pass all the old 1D test cases as a 1D reservoir is just a special case of a 2D reservoir (i.e. with only 1 grid block the $y$ direction).

In [1]:
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import yaml

from assignment13 import OneDimReservoir

In [2]:
class TwoDimReservoir(OneDimReservoir):
    
    #You may want to write some additional functions to parse the inputs, you will need to assign
    #an attribute self.ngrids = N, where N is the total # of grid blocks.
    
    def compute_transmissibility(self, i, j):
        '''
            Computes the transmissibility.
        '''
        k = self.k
        dx = self.delta_x
        dy = self.delta_y
        B = self.formation_volume_factor
        mu = self.viscosity

        #Complete implementation here
        if j == i + 1 or j == i:
            A = np.multiply(dy,self.depth)
            kA_dx =  (2*k[i]*A[i]*k[j]*A[j])/(k[i]*A[i]*dx[j]+k[j]*A[j]*dx[i])
        elif j == i + self.Nx:
            A = np.multiply(dx,self.depth)
            kA_dx =  (2*k[i]*A[i]*k[j]*A[j])/(k[i]*A[i]*dy[j]+k[j]*A[j]*dy[i])

        return (1/B/mu)*kA_dx


    def compute_transmissibility_BC(self, i, j, boundary):
        '''
            Computes the transmissibility.
        '''
        k = self.k
        dx = self.delta_x
        dy = self.delta_y
        B = self.formation_volume_factor
        mu = self.viscosity

        #Complete implementation here
        if boundary == 'left' or boundary == 'right':
            A = np.multiply(dy,self.depth)
            kA_dx =  (2 * k[i] * A[i] * k[j] * A[j]) / (k[i] * A[i] * dx[j] + k[j] * A[j] * dx[i])
        elif boundary == 'top' or boundary == 'bottom':
            A = np.multiply(dx,self.depth)
            kA_dx =  (2 * k[i] * A[i] * k[j] * A[j])/(k[i] * A[i] * dy[j] + k[j] * A[j] * dy[i])

        return (1/B/mu)*kA_dx
    
    
    def compute_accumulation(self, i):
        '''
            Computes the accumulation.
        '''
        volume = self.delta_x[i] * self.delta_y[i] * self.depth
        accumulation = volume * self.phi[i] * self.compressibility / self.formation_volume_factor

        return accumulation #accumulation for the i^{th} grid block
    
    def fill_matrices(self): 
        '''
           Assemble the transmisibility, accumulation matrices, and the flux
           vector.  Returns sparse data-structures
        '''
        #Complete implementation here
        N = self.ngrids
        Nx = self.Nx
        Ny = self.Ny
        Q =  np.zeros(N, dtype=np.double)
        B_matrix = np.zeros((N, N), dtype=np.double)
        T_matrix = np.zeros((N, N), dtype=np.double)
        bcs = self.inputs['boundary conditions']

        for i in range(N):

            if i < N-1:
                #fisll right
                if (i+1)%Nx == 0 and i < N-Nx: 
                    T_matrix[i,i+Nx] = -1 * self.compute_transmissibility(i,i+Nx)
                #filss top row
                elif (i+1)%Nx != 0 and i >= N-Nx: 
                    T_matrix[i,i+1] = -1 * self.compute_transmissibility(i,i+1)
                else: 
                    T_matrix[i,i+1] = -1 * self.compute_transmissibility(i,i+1)
                    T_matrix[i,i+Nx] = -1 * self.compute_transmissibility(i,i+Nx)
                    
            T_matrix[i:N,i] = T_matrix[i,i:N]
            B_matrix[i,i] = self.compute_accumulation(i)
            T_matrix[i,i] = -1 * sum(T_matrix[i,:])


            if i%Nx == 0:
                if 'left' in bcs:
                    if bcs['left']['type'] == 'prescribed pressure':
                        T0 = self.compute_transmissibility_BC(i,i,'left')
                        Q[i] = Q[i] + (2 * T0 * bcs['left']['value'] * self.conversion_factor)
                        T_matrix[i,i] = T_matrix[i,i] + 2 * T0
                    elif bcs['left']['type'] == 'prescribed flux':
                        pass
                    else:
                        pass

            elif (i+1)%Nx == 0:
                if 'right' in bcs:
                    if bcs['right']['type'] == 'prescribed pressure':
                        T0 = self.compute_transmissibility_BC(i,i,'right')
                        Q[i] = Q[i] + (2 * T0 * bcs['right']['value'] * self.conversion_factor)
                        T_matrix[i,i] = T_matrix[i,i] + 2 * T0
                    elif bcs['right']['type'] == 'prescribed flux':
                        pass
                    else:
                        pass

            elif i > N-Nx:
                if 'top' in bcs:
                    if bcs['top']['type'] == 'prescribed pressure':
                        T0 = self.compute_transmissibility_BC(i,i,'top')
                        Q[i] = Q[i] + (2 * T0 * bcs['top']['value'] * self.conversion_factor)
                        T_matrix[i,i] = T_matrix[i,i] + 2 * T0
                    elif bcs['top']['type'] == 'prescribed flux':
                        pass
                    else:
                        pass
            elif i < Nx:
                if 'bottom' in bcs:
                    if bcs['bottom']['type'] == 'prescribed pressure':
                        T0 = self.compute_transmissibility_BC(i,i,'bottom')
                        Q[i] = Q[i] + (2 * T0 * bcs['bottom']['value'] * self.conversion_factor)
                        T_matrix[i,i] = T_matrix[i,i] + 2 * T0
                    elif bcs['bottom']['type'] == 'prescribed flux':
                        pass
                    else:
                        pass


        self.B = scipy.sparse.csr_matrix(B_matrix)
        self.T = scipy.sparse.csr_matrix(T_matrix * self.conversion_factor)
        self.Q = Q

        return 
    
    
    def apply_initial_conditions(self):
        '''
            Applies initial pressures to self.p
        '''

        N = self.ngrids 

        self.p = np.ones(N) * self.inputs['initial conditions']['pressure']

        return

In [3]:
#solver = TwoDimReservoir('inputs.yml')
#solver.solve()
#solver.plot()